# Artificial Intelligence for Trading (Lab Lecture 1)
**Contacts: novella@di.uniroma1.it, coletta@di.uniroma1.it, piva@di.uniroma1.it, prata@di.uniroma1.it**

## Introduction to Pandas
In this first part of the course, you will learn to **manipulate financial data with Python**. 

We will start by handling **pandas** (<a href="https://pandas.pydata.org/docs/">doc</a>), which is a Python package providing fast, flexible, and expressive data structures designed to make working
with “relational” or “labeled” data both easy and intuitive. It aims to be a high-level building
block for doing practical, real world **data analysis** in Python.

The two primary data structures of pandas, **Series** (1-dimensional) and **DataFrame** (2-dimensional), handle
the vast majority of typical use cases in finance, statistics, social science, and many areas of engineering.
pandas is built on top of NumPy and is intended to integrate well within a scientific computing environment.

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.width = 1200

A **DataFrame** is a 2-dimensional data structure that can store data of different types (including characters,
integers, floating point values, categorical data and more) in columns. It is similar to a spreadsheet, a
SQL table or the data.frame in R. Let us create one and print it.

In [ ]:
# creating a dataframe by columns
data_cols = {"Name":["Alice", "Bob", "Claire", "Daniel"], "Age":[12, 23, 44, 59]}  # as a dictionary, column wise
df = pd.DataFrame(data_cols)
print(df)

# equivalently by rows
data_rows = [("Alice", 12), ("Bob", 23), ("Claire", 44), ("Daniel", 59)]  # as a list, row wise
df = pd.DataFrame(data_rows, columns=['Name','Age'])
print(df)

We will be dealing with data coming from **CSV** (**C**omma **S**eparated **V**alues) files. Here is an example of a CSV dataset of videos trending on Youtube.

<img src="data/press_images/YT_table.png"/>

## Manipulating Real Stock Market Dataset
You can download real stock market dataset at https://finance.yahoo.com. They will look like the picture below (**AAPL.csv**). 

<img src="data/press_images/AAPL_table.png" width="600" height="700"/>

Notice: a trading day starts at 9:30 a.m. and ends at 4:00 p.m.. There is no trading activity during weekends and holidays. 


In [ ]:
!ls data

In [ ]:
def test_01():
    """ Print the whole dataframe. """
    df = pd.read_csv("data/AAPL.csv")
    print(df)

test_01()

In [ ]:
def test_02():
    """ Print the first 5 lines of a dataframe. """
    df = pd.read_csv("data/AAPL.csv")
    print(df.head())

test_02()

In [ ]:
def test_03():
    """ Print a slice of a dataframe. """
    df = pd.read_csv("data/AAPL.csv")
    
    # Access a group of rows and columns by integer index
    print(df.iloc[10:21])  # rows between index 10 and 20 (by slicing)

test_03()

In [ ]:
!ls data

In [ ]:
def get_max_adjclose(symbol):
    """ Returns the max value of CLOSE of the symbol. """
    df = pd.read_csv("data/%s.csv" % symbol)
    return df['Adj Close'].max()

def test_04():
    """ Print some symbol names along with the max adj closing price for that symbol. """
    for symbol in ['AAPL', 'TSLA', 'MSFT']:
        print("Max adj close", symbol, get_max_adjclose(symbol))

test_04()

In [ ]:
def test_05():
    """ Plot the adjusted close for Apple. """
    df = pd.read_csv("data/AAPL.csv")
    df['Adj Close'].plot()
    plt.show()
    
test_05()

In [ ]:
def test_06():
    """ Plot high and low for Tesla. """
    df = pd.read_csv("data/TSLA.csv")
    df[['High', 'Low']].plot(title="TSLA Stock")
    plt.show()

test_06()

In [ ]:
def test_07():
    """ Assign an index to a new dataframe. """
    # dates is formatted as YYYY-MM-DD
    
    start_date = '2019-10-09'
    end_date = '2020-02-01'
    dates = pd.date_range(start_date, end_date)
    print(dates) # DatetimeIndex object with timestamps
     
    df = pd.DataFrame(index=dates) # the dataframe only with the index
    print(df)

test_07()

In [ ]:
def test_08():
    """ Join an empty DataFrame with index, with the Apple Dataframe. """

    # dates is formatted as YYYY-MM-DD
    start_date = '2019-10-09'
    end_date = '2020-02-01'
    dates = pd.date_range(start_date, end_date)

    # An empy dataframe
    df1 = pd.DataFrame(index=dates) # the dataframe only with the index
    
    # join with Apple stock, attempt 1
    dfAAPL = pd.read_csv("data/AAPL.csv")
    df1 = df1.join(dfAAPL)  # join by the index by defult
    
    # join with Apple stock, attempt 2
    # dfAAPL = pd.read_csv("data/AAPL.csv", index_col="Date", parse_dates=True) 
    # df1 = df1.join(dfAAPL)

    # join with Apple stock, attempt 3
    # dfAAPL = pd.read_csv("data/AAPL.csv", index_col="Date", parse_dates=True) 
    # # {‘left’, ‘right’, ‘outer’, ‘inner’}, default ‘left’
    # df1 = df1.join(dfAAPL, how="inner") 

    # join with Apple stock, attempt 4
    # dfAAPL = pd.read_csv("data/AAPL.csv", index_col="Date", parse_dates=True, usecols=["Date", "Adj Close"])
    # df1 = df1.join(dfAAPL, how="inner")
    
    print(df1)

test_08()

In [ ]:
def path_to_symbol(symbol_name, root_symbols="data"):
    """Returns the path to the symbol. """
    return os.path.join(root_symbols, "%s.csv" % symbol_name)

def adjusted_close_symbols(symbols, dates=('2019-10-09', '2020-10-09')):
    """ Joins an empty DataFrame with index, with multiple stocks dataframes. """

    # dates is formatted as YYYY-MM-DD
    start_date, end_date = dates
    dates = pd.date_range(start_date, end_date)

    # An empty dataframe
    df1 = pd.DataFrame(index=dates) # the dataframe only with the index

    for sym in symbols:
        dfSYM = pd.read_csv(path_to_symbol(sym), index_col="Date", parse_dates=True,
                            usecols=["Date", "Adj Close"], na_values=['nan'])
        dfSYM = dfSYM.rename(columns= {'Adj Close': sym})
        df1 = df1.join(dfSYM, how="inner")

    return df1

def test_09():
    df = adjusted_close_symbols(["^GSPC", "AAPL", "TSLA", "FB", "MSFT"])
    print(df)

test_09()

In [ ]:
def test_10():
    """ Slicing. """
    df = adjusted_close_symbols(["^GSPC", "AAPL", "TSLA", "FB", "MSFT"])

    # Slicing over the rows, loc: access a group of rows and columns by label(s) or a boolean array.
    df1 = df.loc['2020-03-01':'2020-03-31']  # iloc for integer indices, loc for labels
    print(df1)

    # Slicing over the columns
    df2 = df[['^GSPC', 'AAPL']]  # Notice no .loc here
    print(df2)

    # Slicing over both dimensions
    df2 = df.loc['2020-03-01':'2020-03-31', ['^GSPC', 'AAPL']]
    print(df2)

test_10()

In [ ]:
def plot_dataframe(df, title, xlabel, ylabel):
    """It will plot the dataframe, given a plot tile and labels for the axes. """
    ax = df.plot(title=title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.show()

def test_11():
    """ Normalising and plotting multiple stock prices. """
    df = adjusted_close_symbols(["^GSPC", "AAPL", "FB", "MSFT"])
    plot_dataframe(df, "Adjusted Closes", "Date", "Price")
    print(df)

    # What to do if you want all the stocks to start at 1.0?
    # We want to divide by day 1
    df_norm = df / df.iloc[0,:]
    plot_dataframe(df_norm, "Adjusted Closes", "Date", "Price")

test_11()

### Statistics Over Time Series
Pandas makes it easy to compute convenient global statistics on a dataframe: **mean, median, std, sum** and
<a href="https://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-stats">more</a>; and rolling
statistics: **rolling mean, rolling std** and <a href="http://pandas.pydata.org/pandas-docs/stable/computation.html?highlight=rolling%20statistics#moving-rolling-statistics-moments">more</a>.

In [ ]:
def test_12():
    """ Print global statistics [mean]. """
    df = adjusted_close_symbols(["^GSPC", "AAPL", "FB", "MSFT"])
    print(df.mean())

test_12()

An example of rolling mean.

<img src="data/press_images/rolling_mean.gif" hight="500" width="500"/>

In [ ]:
def test_13():
    """ Print rolling statistics [mean]. We take snapshots of the statistic over time windows, and save it,
    move the window of one unit and star again. """

    df = adjusted_close_symbols(["AAPL"])

    window_size = 20  # days
    aapl_rm = df.rolling(window_size).mean()
    
    ax = aapl_rm.plot()
    df.plot(ax=ax)
    ax.axhline(y=df.mean()[0], color='r', linestyle='-', label="global mean")
    
    plt.legend()
    plt.show()

test_13()

## Introduction to NumPy
**NumPy** (<a href="https://numpy.org/doc/1.19/">doc</a>) is an open source project aiming to enable numerical computing with Python. NumPy is developed in
the open on GitHub, through the consensus of the NumPy and wider scientific Python community.

NumPy’s main object is the **homogeneous multidimensional array**. It is a table of elements (usually numbers),
all of the same type, indexed by a tuple of non-negative integers. In NumPy dimensions are called axes.
NumPy’s array class is called ndarray. It is also known by the alias array.

It is related to pandas as a DataFrame is a wrapper of an ndarray. Treating data as ndarrays rather than
just DataFrames will give you access to many additional routines.

In [ ]:
print(pd.read_csv("data/MSFT.csv").head())

In [ ]:
import numpy as np

# creating ndarrays
d1_ndarray = np.array([1,2,3])  # 1D
d2_ndarray = np.array([[1,2,3], [4,5,6], [7,8,9]])  # 2D

print(d1_ndarray)
print(d2_ndarray)

In [ ]:
# init ndarrays
d2_zero = np.zeros((3,3))  # also ones!

print(d2_zero)

In [ ]:
d2_rand_int = np.random.randint(0,100,(4,4))
d1_rand = np.random.rand(4,4)

print(d2_rand_int)
print(d1_rand)
print(d1_rand.shape, d1_rand.size, d1_rand.dtype)

# you can also sample from different distribution other than uniform

In [ ]:
def test_14():
    """ Extract an ndarray from a DataFrame object and play with slicing. """

    df = adjusted_close_symbols(["^GSPC", "AAPL", "FB", "MSFT"], ('2020-10-01', '2020-10-09'))
     
    nd1 = df.values
    print(nd1, type(nd1))

    # access a value
    print("Element in 0,0:", nd1[0,0])  # [row, column]
    print("Element in 2,1:", nd1[2,1])  # [row, column]
    
    # access slices
    print(nd1[0:3,1:3])  # [row, column]

    # access rows and columns
    print("Row 0:", nd1[0,:])
    print("Column 0:", nd1[:,0])

    # negative indices
    print(nd1[-1, 1:3])

    # Replace slices
    nd1[0:3,0:2] = nd1[3:,2:]
    print(nd1)

test_14()

In [ ]:
def test_15():
    """ Operations over axes. """

    np.random.seed(199)  # seed the random generator
    d2_rand_int = np.random.randint(0,100,(3,7))
    print(d2_rand_int) 

    print("Sum of each column:\n", d2_rand_int.sum(axis=0))  # sum the vales on the rows, encoded as 0
    print("Sum of each row:\n", d2_rand_int.sum(axis=1))  # sum the vales on the rows, encoded as 0

    print("Minimum of each row:\n", d2_rand_int.min(axis=0))
    print("Maximum of each row:\n", d2_rand_int.min(axis=1))
    print("Mean of all:\n", d2_rand_int.mean())  # axis 1 = what to iterate first

test_15()

In [ ]:
import time

def manual_sum(arr):
    sum = 0
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            sum += arr[i,j]
    return sum

def test_16():
    """ Timing numpy. """

    np.random.seed(199)  # seed the random generator
    d1_rand = np.random.rand(2000, 2000)

    t1 = time.time()
    print(np.sum(d1_rand))
    t2 = time.time()
    elapsed_np = t2 - t1

    t1 = time.time()
    print(manual_sum(d1_rand))
    t2 = time.time()
    elapsed_man = t2 - t1

    print("Seconds elapsed for numpy", elapsed_np)
    print("Seconds elapsed for manual", elapsed_man)

test_16()

In [ ]:
def test_17():
    """ Indexing. """

    d1_rand = np.random.rand(20)
    indices = np.array([0,10,15,19])
    print(d1_rand)
    print(d1_rand[indices])

    # select all the elements lower than the mean of array
    mean = d1_rand.mean()
    print(mean)
    print(d1_rand<mean)  # mask

    d1_rand = d1_rand[d1_rand<mean]
    print(d1_rand)

test_17()

In [ ]:
def test_18():
    """ Operations. """

    a = np.array([[1,2,3,4,5], [10,20,30,40,50]])
    b = np.array([[100,200,300,400,500], [1,2,3,4,5]])
    print("A matrix\n", a)
    print("B matrix\n", b)
    
    # element wise multiplication 
    print("Multiplication by 2\n", 2*a)

    # element wise division 
    print("Division by 2\n", a / 2)
    
    # element wise sum
    print("Sum a and b\n", a+b)

    # element wise multiplication
    print("Multiplication a and b\n", a*b)

    # matrix multiplication
    print("Mat mul a and b\n", np.matmul(a, np.transpose(b)))
    
    # When the dataframe contains only numbers (exclusing hedar and index) and have the same index
    # you can use dataframes as if they were ndarrays 
            
test_18()

This notebook and more is available for download at: https://github.com/matteoprata/AI4Trading_20_21